# Python with NoSQL Exercise 1

You may need to do this on a local environment with MongoDB server and pymongo installed.

A MongoDB database has a flexible schema. Create a MongoDB database business with a collection orders.

In [ ]:
from pymongo import MongoClient

#create mongo clientt
client = MongoClient('localhost',27017)
my_db = client['orders']
mycol = my_db['items']

The data file orders_sample.json shows some possible documents. Comment on the current schema. What can you say about the differences between this and the SQL context?

Generate a suitable orders.json.

Answer: SQL context makes use of a relational database schema. NoSQL use a document style collection to store data. The documents in the sample_orders include fields from the Customers, Products as well as orders document, instead of being placed in separate schemas

In [ ]:
import csv
import json

In [ ]:
order_data = dict()
#sample
#orderid, customerid,customername,contactno,orderdate,orderdetails(productid,productname,unitprice,quantity)

def csv_to_json(filepath,mainkey):
    with open(filepath) as file:
        reader = csv.DictReader(file)
        #add each row as a new dictionary
        fin = []
        for row in reader:
            fin.append(row)
    
    with open(f'{filepath[:-4]}.json','w') as jsonf:
        jsonf.write(json.dumps(fin, indent=4))
    return fin
csv_to_json('orders.csv','OrderID')
csv_to_json('products.csv','ProductID')
csv_to_json('order_details.csv','OrderID')
csv_to_json('customers.csv','CustomerID')

Insert the documents from your generated orders.json into the orders collection.

Write query statements to determine:
- number of orders made on a particular day entered by a user
- total revenue on a particular day
- number of orders made by a particular customer
- product(s) with lowest sales
- best selling product(s) over a period of time (start to end dates inclusive)

In [ ]:
import pymongo
from pymongo import MongoClient
import json

#create mongo client
client = MongoClient('localhost',27017)
db_store = client['store']

#CustomerID	CustomerName	ContactNo
db_customers = db_store['customers']

#OrderID	ProductID	Quantity
db_orderdetails = db_store['order_details'] 

#ProductID	ProductName	UnitPrice
db_products = db_store['products'] 

#OrderID	CustomerID	OrderDate
db_orders = db_store['orders'] 

In [ ]:
#load documents
def load_documents(mongocol,filepath):
    with open(filepath) as f:
        file_data = json.load(f)
        x = mongocol.insert_many(file_data)
        return x.inserted_ids
        '''
        for line in f:
            #file_data = json.loads(line)
            x = mycol.insert_one(file_data)
            print(x.inserted_ids)'''

load_documents(db_customers,'customers.json')
load_documents(db_orderdetails,'order_details.json')
load_documents(db_products,'products.json')
load_documents(db_orders,'orders.json')

In [ ]:
#remove duplicates by OrderIDs and groups them
#"_id" is a necessary field for mongodb
new_orderdetails = db_orderdetails.aggregate([
    {"$group":{
        "_id":"$OrderID",
        "OrderID":{"$first":"$OrderID"},
        "Orders_made":{"$push":{
            "ProductID": "$ProductID",
            "Quantity": "$Quantity"
        }}
    }} 
])
#print(f"After:{db_orderdetails.count()}")
new_ordersd = {}
#prevent similar items from being overrided in dictionary
for count,item in enumerate(new_orderdetails):
    new_ordersd[count] = item
    print(item)

#output
'''
{'_id': '10533', 'OrderID': '10533', 'Orders_made': [{'ProductID': '4', 'Quantity': '50'}, {'ProductID': '72', 'Quantity': '24'}, {'ProductID': '73', 'Quantity': '24'}]}
{'_id': '10788', 'OrderID': '10788', 'Orders_made': [{'ProductID': '19', 'Quantity': '50'}, {'ProductID': '75', 'Quantity': '40'}]}
...
'''

In [ ]:
#Task 1
#print(new_ordersd)
customer_id = 'TOMSP'
day = '1996-07-05'
cust_id = db_orders.find_one({"CustomerID":customer_id,"OrderDate":day},{"OrderID"})
print(cust_id['OrderID'])
for k in range(len(new_ordersd)):
    if new_ordersd[k]['OrderID'] == cust_id['OrderID']:
        print(new_ordersd[k]['Orders_made'])
        print(f"Number of orders made: {len(new_ordersd[k]['Orders_made'])}")


10249
[{'ProductID': '14', 'Quantity': '9'}, {'ProductID': '51', 'Quantity': '40'}]
Number of orders made: 2


In [ ]:
#Task 2
rev_day = '1996-08-14' #Day to count
bought_products = list(db_orders.find({"OrderDate":rev_day},{"OrderID"})) #find days that match
print(bought_products)

total_revenue = 0

#iterate through each product bought
for i in range(len(bought_products)):
    #checking with existing orders dictionary (order_details)
    for k in range(len(new_ordersd)):
        #found corresponding IDs
        if new_ordersd[k]['OrderID'] == bought_products[i]['OrderID']:
            for m in range(len(new_ordersd[k]['Orders_made'])):
                #add each quantity for the order
                pid = new_ordersd[k]['Orders_made'][m]['ProductID']
                qty =  int(new_ordersd[k]['Orders_made'][m]['Quantity'])
                #Filter,projection
                price = db_products.find_one({"ProductID":pid},{"UnitPrice"})
                price = float(price["UnitPrice"])
                print(f"price {price}, qty {qty}")
                total_revenue+=(qty*price)
print(f"TotalRevenue {total_revenue}")

[{'_id': ObjectId('5eec9ee0169c0f1347b9b144'), 'OrderID': '10280'}, {'_id': ObjectId('5eec9ee0169c0f1347b9b145'), 'OrderID': '10281'}]
price 4.5, qty 12
price 24.0, qty 20
price 7.75, qty 30
price 9.2, qty 1
price 4.5, qty 6
price 18.0, qty 4
TotalRevenue 874.7


In [ ]:
#Task 3
b_customer = 'TOMSP'
print("No. of orders made: ",db_orders.count_documents({"CustomerID":b_customer}))

No. of orders made:  6


In [ ]:
#Task 4
#remove duplicates by product IDs and groups them
#"_id" is a necessary field for mongodb
productsales = db_orderdetails.aggregate([
    {"$group":{
        "_id":"$ProductID",
        "ProductID":{"$first":"$ProductID"},
        "Orders_list":{"$push":{
            "OrderID": "$OrderID",
            "Quantity": "$Quantity"
        }}
    }} 
])

new_productsales = {}

#adds an index to append each item uniquely to prevent overriding 
for count,item in enumerate(productsales):
    new_productsales[count] = item
    #print(item)

total_orders = dict()
for i in range(len(new_productsales)):
    total_orders[new_productsales[i]['ProductID']] = len(new_productsales[i]['Orders_list'])
print(dict(sorted(total_orders.items(), key=lambda x: x[1], reverse=False)))
print(f"Most Unpopular Product has ID {list(dict(sorted(total_orders.items(), key=lambda x: x[1], reverse=False)).keys())[0]}")

{'9': 5, '48': 6, '37': 6, '15': 6, '66': 8, '27': 9, '67': 10, '5': 10, '50': 10, '3': 12, '6': 12, '8': 13, '74': 13, '12': 14, '45': 14, '22': 14, '73': 14, '32': 15, '20': 16, '63': 17, '58': 18, '25': 18, '34': 19, '4': 20, '23': 20, '49': 21, '47': 21, '14': 22, '57': 23, '61': 24, '44': 24, '38': 24, '18': 27, '46': 27, '43': 28, '7': 29, '52': 29, '39': 30, '42': 30, '53': 30, '64': 30, '36': 31, '69': 31, '65': 32, '26': 32, '33': 32, '30': 32, '29': 32, '55': 33, '28': 33, '10': 33, '68': 34, '54': 36, '35': 36, '19': 37, '17': 37, '72': 38, '77': 38, '1': 38, '11': 38, '76': 39, '70': 39, '21': 39, '51': 39, '13': 40, '40': 41, '71': 42, '16': 43, '2': 44, '75': 46, '41': 47, '62': 48, '56': 50, '60': 51, '24': 51, '31': 51, '59': 54}
Most Unpopular Product has ID 9


In [ ]:
#Task 5
start_date = '1996-07-04'
end_date = '1996-07-08'

orders_made = list(db_orders.find({"OrderDate":{"$gte":start_date,"$lte":end_date}},{"OrderID"}))
print(orders_made)

bestselling = dict()
for i in range(len(orders_made)):
    print(orders_made[i]["OrderID"])
    
    #find dates in ORder_details with same quantity and productid
    temp = list(db_orderdetails.find({"OrderID":{"$eq":orders_made[i]["OrderID"]}},{"ProductID","Quantity"}))
    for j in range(len(temp)):
        #print(temp[j]['ProductID'])
        bestselling[temp[j]["ProductID"]] = int(temp[j]["Quantity"])
        
#sort dictionary        
print(dict(sorted(bestselling.items(),key=lambda x:x[1],reverse=True)))
best = list(dict(sorted(bestselling.items(),key=lambda x:x[1],reverse=True)).keys())[0]
print(f"Most Popular Product from {start_date} to {end_date} is {best}")

[{'_id': ObjectId('5eec9ee0169c0f1347b9b124'), 'OrderID': '10248'}, {'_id': ObjectId('5eec9ee0169c0f1347b9b125'), 'OrderID': '10249'}, {'_id': ObjectId('5eec9ee0169c0f1347b9b126'), 'OrderID': '10250'}, {'_id': ObjectId('5eec9ee0169c0f1347b9b127'), 'OrderID': '10251'}]
10248
10249
10250
10251
{'51': 35, '65': 20, '57': 15, '11': 12, '42': 10, '41': 10, '14': 9, '22': 6, '72': 5}
Most Popular Product from 1996-07-04 to 1996-07-08 is 51
